<a href="https://colab.research.google.com/github/filipecalegario/stylegan2-ada-experiments/blob/main/Pr%C3%A9_processamento_de_Imagens_StyleGan2_ADA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# StyleGAN2-ADA (PyTorch): Pré-processando imagens para treinar um modelo

Referência: https://github.com/NVlabs/stylegan2-ada-pytorch

## Configurações Iniciais

### Checar uso de GPU (placa de vídeo)

Ao executar a célula a seguir, é esperado que você veja uma tabela com as configurações da placa de vídeo atualmente utilizada por seu ambiente de execução.
<br>
Caso o comando a seguir falhe, vá no menu do canto superior esquerdo e, na aba ***Runtime***, entre em ***Change runtime type***. Dentro desse novo menu, vá em ***Hardware accelerator*** e selecione a opção ***GPU***, por fim salve.
<br>
Rode novamente a célula abaixo para checar se o ambiente de execução agora tem uma placa de vídeo conectada.

In [ ]:
!nvidia-smi

### Instalação de pré-requisitos

Referência: https://github.com/NVlabs/stylegan2-ada-pytorch/issues/2

In [ ]:
%pip install ninja
%pip install opencv-python

### Baixar código do StyleGAN2-ADA (PyTorch)

Referência: https://github.com/woctezuma/stylegan2-ada-pytorch/tree/google-colab 
<br>
Fork utilizada: https://github.com/ClaudioCarvalhoo/stylegan2-ada-pytorch

In [ ]:
%cd /content/

%rm -rf stylegan2-ada-pytorch/
!git clone https://github.com/ClaudioCarvalhoo/stylegan2-ada-pytorch

%cd stylegan2-ada-pytorch/

%cd /content/

## Conectar com Google Drive

In [ ]:
%pip install Google-Colab-Transfer

Ao executar a próxima célula, você receberá um link para pegar seu código de verificação do Google Drive.
<br>
Acesse com sua conta que tem as imagens de treinamento no Drive e cole o código recebido na caixa de texto que aparecerá abaixo.

In [ ]:
import colab_transfer

colab_transfer.mount_google_drive()

## Normalizando Imagens

Após executar a próxima célula, insira nas caixas de texto o caminho para a pasta do seu Drive que contém as imagens a serem utilizadas para treinar seu modelo e também o caminho para a pasta do seu Drive na qual deseja salvar as imagens após serem processadas.
<br>
Lembre-se de manter intacta a parte inicial do texto (/content/drive/MyDrive/) pois é o caminho do Colab para acessar a raíz de seu Drive.

In [ ]:
import ipywidgets as widgets

custom_images = widgets.Text(
    value='/content/drive/MyDrive/sua_pasta/imagens_originais',
    placeholder='Digite ou cole aqui o caminho',
    layout=widgets.Layout(width='50%')
)
dataset = widgets.Text(
    value='/content/drive/MyDrive/sua_pasta/dataset',
    placeholder='Digite ou cole aqui o caminho',
    layout=widgets.Layout(width='50%')
)
widgets.VBox([
    widgets.VBox([widgets.Label('Caminho para as imagens originais:'), custom_images]),
    widgets.VBox([widgets.Label('Caminho para salvar as imagens processadas:'), dataset])          
])

Após executar a próxima célula, escolha um tamanho (em pixels) para a altura e largura das imagens que serão geradas. Quanto maior, mais demorado será o treinamento da rede.
<br>
Você pode também escolher a maneira com a qual serão geradas bordas para deixar as imagens quadradas. Caso não saiba o que cada um faz, pode deixar no valor padrão!

In [ ]:
import cv2

IMG_SIZE = widgets.ToggleButtons(
    options=[128, 256, 512, 1048],
    value=256
)
BORDER_TYPE = widgets.Dropdown(
    options=[('BORDER_REPLICATE', cv2.BORDER_REPLICATE), ('BORDER_CONSTANT', cv2.BORDER_CONSTANT), ('BORDER_REFLECT', cv2.BORDER_REFLECT), ('BORDER_WRAP', cv2.BORDER_WRAP), ('BORDER_REFLECT_101', cv2.BORDER_REFLECT_101), ('BORDER_TRANSPARENT', cv2.BORDER_TRANSPARENT), ('BORDER_REFLECT101', cv2.BORDER_REFLECT101), ('BORDER_DEFAULT', cv2.BORDER_DEFAULT), ('BORDER_ISOLATED', cv2.BORDER_ISOLATED)],
    value=cv2.BORDER_REPLICATE,
)

widgets.VBox([
    widgets.Label('Tamanho das imagens geradas:'),
    IMG_SIZE,
    widgets.Label('Estilo das bordas:'),
    BORDER_TYPE
])


A célula a seguir processará todas as suas imagens deixando-as quadradas e do mesmo tamanho (requisitos obrigatórios para treinamento do StyleGan).

In [ ]:
import numpy as np
import os

def getBorderSizes(size):
    necessaryPadding = IMG_SIZE.value - size
    borderSize = necessaryPadding // 2
    if necessaryPadding % 2 == 0:
        return [borderSize, borderSize]
    else:
        return [borderSize + 1, borderSize]

def getDirFiles(dir):
  return [f for f in os.listdir(dir) if os.path.isfile(f"{dir}/{f}")]

processedImagesDir = f"{custom_images.value}/processed"
processed = set()
if "processed" in os.listdir(f"{custom_images.value}"):
  processed = set(
      [filename.split(".")[0] for filename in getDirFiles(processedImagesDir)]
  )
else:
  os.mkdir(f"{custom_images.value}/processed")

def processImage(filename):
    if len(filename.split(".")) <= 1 or filename.split(".")[0] in processed:
        return

    img = cv2.imread(f"{custom_images.value}/{filename}")

    f1 = IMG_SIZE.value / img.shape[1]
    f2 = IMG_SIZE.value / img.shape[0]
    f = min(f1, f2)  # resizing factor
    dim = (int(img.shape[1] * f), int(img.shape[0] * f))

    img = cv2.resize(img, dim)

    heightBorders = getBorderSizes(img.shape[0])
    widthBorders = getBorderSizes(img.shape[1])

    img = cv2.copyMakeBorder(
        img,
        top=heightBorders[0],
        bottom=heightBorders[1],
        left=widthBorders[0],
        right=widthBorders[1],
        borderType=BORDER_TYPE.value,
    )

    cv2.imwrite(f"{processedImagesDir}/{filename}.jpg", img)
    print(f"Processed: {filename}")

for img in getDirFiles(custom_images.value):
  processImage(img)

## Pré-processando Imagens

A célula a seguir transformará suas imagens em um conjunto de dados capaz de ser utilizado para o treinamento da rede.

Referência: https://github.com/NVlabs/stylegan2-ada-pytorch#preparing-datasets

In [ ]:
!python stylegan2-ada-pytorch/dataset_tool.py \
 --source {processedImagesDir} \
 --dest {dataset.value}